In [1]:
import pandas as pd
import psycopg2
import numpy as np
pd.set_option('display.max_columns', None) # or 1000.

In [2]:
config = {
    'DBNAME':'adsdb',
    'USER':'postgres',
    'PASSWORD':'postgresadmin',
    'SCHEMA':'formatted'
}

In [3]:
conn = psycopg2.connect(dbname=config['DBNAME'], user=config['USER'], password=config['PASSWORD'])

In [4]:
df_tables = pd.read_sql('SELECT table_name FROM information_schema.tables WHERE table_schema=\'' + config['SCHEMA'] +'\'   AND table_type=\'BASE TABLE\';', conn)
df_tables

,table_name
0,morticd_10_1_f695_1642950107
1,morticd_10_3_1c6a_1642928618
2,morticd_10_4_453e_1642928618
3,notes_0_a67b_1642928619
4,population_0_3b8f_1642928619
5,demographiccountry_0_e0e0_1642950106
6,demographic_0_76f7_1642950106
7,demographiclabels_0_e8a5_1642950107
8,morticd_7_22b4_1642950107
9,morticd_8_9366_1642950107


In [5]:
max_timestamp = df_tables['table_name'].apply(lambda x: x[x.rfind('_') +1 :-2]).max()
max_timestamp

'16429501'

In [6]:
df_lastupdate_tables = df_tables['table_name'].apply(lambda x: max_timestamp in x)
df_tables[df_lastupdate_tables]

,table_name
0,morticd_10_1_f695_1642950107
5,demographiccountry_0_e0e0_1642950106
6,demographic_0_76f7_1642950106
7,demographiclabels_0_e8a5_1642950107
8,morticd_7_22b4_1642950107
9,morticd_8_9366_1642950107
10,morticd_9_71c9_1642950107
11,morticd_10_2_ffb1_1642950108
12,morticd_10_3_1c6a_1642950108
13,morticd_10_4_453e_1642950109


In [7]:
mortic_dataframes = []
demo_dataframes = []
general_dataframes = []
for table in df_tables[df_lastupdate_tables]['table_name']:
    firstDash = str(table).index('_', 0)
    lastDash = str(table).rindex('_')
    secondlastDash = str(table).rindex('_', lastDash)
    dfname = str(table[:secondlastDash])
    if  dfname.count('_') >= 2:
        lastDash = dfname.rindex('_')
        dfname = dfname[:lastDash]
    if 'morticd' in dfname:
        mortic_dataframes.append('df_' + dfname)
    elif 'demo' in dfname:
        demo_dataframes.append('df_' + dfname)
    else:
        general_dataframes.append('df_' + dfname)
    globals()['df_' + dfname] = pd.read_sql('SELECT * from ' + config['SCHEMA'] + '.' + table + ';', conn)

In [8]:
demo_dataframes

['df_demographiccountry_0', 'df_demographic_0', 'df_demographiclabels_0']

In [9]:
mortic_dataframes

['df_morticd_10_1',
 'df_morticd_7',
 'df_morticd_8',
 'df_morticd_9',
 'df_morticd_10_2',
 'df_morticd_10_3',
 'df_morticd_10_4']

In [10]:
general_dataframes

['df_countrycodes_0', 'df_notes_0', 'df_population_0']

# Outliers

The following row indexes have been found as outliers by the Boxplot quantiles

In [46]:
for df in mortic_dataframes:
    df_num = globals()[str(df)].select_dtypes(include=np.number)
    imp = SimpleImputer(strategy='median')
    df_num = imp.fit_transform(df_num)
    q1 = np.quantile(df_num, 0.25)

    # finding the 3rd quartile
    q3 = np.quantile(df_num, 0.75)
    med = np.median(df_num)
    
    # finding the iqr region
    iqr = q3-q1
    
    # finding upper and lower limits
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    outliers = df_num[(df_num <= lower_bound) | (df_num >= upper_bound)]
    print('The following are the outliers for ' + df + ' in the boxplot:{}'.format(outliers))
    

The following are the outliers for df_morticd_10_1 in the boxplot:[1125. 2000. 1125. ... 5150. 2002.    8.]
The following are the outliers for df_morticd_7 in the boxplot:[  1125.   1955. 107822. ...   1960.   4340.   1960.]
The following are the outliers for df_morticd_8 in the boxplot:[1060. 1980. 1087. ...  241.  130.   71.]
The following are the outliers for df_morticd_9 in the boxplot:[2005. 1983.   36. ...   15.    9.    6.]
The following are the outliers for df_morticd_10_2 in the boxplot:[1400. 2003.  377. ... 4240. 2005.  329.]
The following are the outliers for df_morticd_10_3 in the boxplot:[1400. 2008.  392. ...    7.    3.    3.]
The following are the outliers for df_morticd_10_4 in the boxplot:[1400. 2013.  383. ...    6. 5020. 2016.]
